In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generate_rotations(x):
    #function for givens rotation, takes a vector x and returns a vector with the accoring sin and cosinee vales
    s = np.zeros_like(x)
    c = np.ones_like(x)
    for i in range(x.size-2,-1,-1): # loop from second-last element to index 0
        a = x[i]
        b = x[i+1]
        if a ==0:
            s[i] =-1
            c[i] =0
        else:
            
            tau = -b/a
            c[i] = 1/np.sqrt(1+tau**2)
            s[i] = tau*c[i]
        x[i]  = c[i]*a-s[i]*b
        x[i+1]= s[i]*a+c[i]*b
    return [s,c,x]

def apply_rotations(s,c,x):
    for i in range(x.size-2,-1,-1): # loop from second-last element to index 0
        a = x[i]
        b = x[i+1]
        x[i]  = c[i]*a-s[i]*b
        x[i+1]= s[i]*a+c[i]*b
    return [x]

In [ ]:
x = np.random.rand(5)
x_prime = x.copy()

In [ ]:
[s,c,x]=generate_rotations(x)
x

In [ ]:
apply_rotations(s,c,x_prime)
x

In [ ]:
#generate a strict system
#We will generate some stages here. 
#all C matrices will have 5*3 shape
#and B accoringly 3*5
N=5
stages = [Stage(np.random.rand(3*(n<N-1),3*(n>0)),np.random.rand(3*(n<N-1),5),np.random.rand(5,3*(n>0)),np.random.rand(5,5)) for n in range(N)]
syst = StrictSystem(causal=True,stages=stages)

In [ ]:
#short test
inp = np.random.rand(25,3)
syst.compute(inp)

In [ ]:
A = syst.to_matrix()
plt.spy(A)

In [ ]:
#now run the first step of the QR decomposition
S = np.zeros((25,3))
C = np.zeros((25,3))
for j in range(3):
    [s,c,x]=generate_rotations(A[j:,j])
    S[j:,j]=s
    C[j:,j]=c
    for i in range(j+1,25):
        apply_rotations(s,c,A[j:,i])


In [ ]:
plt.spy(abs(A)>1e-10)

In [ ]:
A[:,14]

In [ ]:
np.random.rand(5,0)

Now we consider how it behaves if the lowest C is not fully used in the first collumn

In [ ]:
#generate a strict system
#We will generate some stages here. 
#all C matrices will have 5*3 shape
#and B accoringly 3*5
N=5
stages2 = [Stage(np.random.rand(3*(n<N-1),3*(n>0)),np.random.rand(3*(n<N-1),5),np.random.rand(5,3*(n>0)),np.random.rand(5,5)) for n in range(N)]
stages2[4].A_matrix[:,0]=0
syst2 = StrictSystem(causal=True,stages=stages2)

In [ ]:
A2 = syst2.to_matrix()

In [ ]:
#now run the first step of the QR decomposition
S = np.zeros((25,3))
C = np.zeros((25,3))
for j in range(3):
    [s,c,x]=generate_rotations(A2[j:,j])
    S[j:,j]=s
    C[j:,j]=c
    for i in range(j+1,25):
        apply_rotations(s,c,A2[j:,i])

In [ ]:
plt.matshow(np.log(np.abs(A2)+1e-15))